# Train GPT-2 Medium with RLAIF
Steps to run:
* Add the following to secret keys: Github PAT, HF Token (write access), CO_API_KEY


In [1]:
import os
import sys
from google.colab import userdata

# ADD A PAT https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
os.environ['GITHUB_TOKEN'] = userdata.get('GITHUB_TOKEN')  # put the token in colab secret keys
os.environ['REPOSITORY'] =  "github.com/jth500/maet-pln.git"
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
!git clone -b refactor https://${{GITHUB_TOKEN}}@${{REPOSITORY}}
!mv maet-pln maet_pln # rename to remove the invalid dash
sys.path.append("maet_pln/src")

Cloning into 'maet-pln'...
remote: Enumerating objects: 535, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 535 (delta 59), reused 80 (delta 41), pack-reused 416
Receiving objects: 100% (535/535), 601.57 KiB | 15.42 MiB/s, done.
Resolving deltas: 100% (332/332), done.


In [ ]:
!pip install -q pyarrow==12.0.0
# !pip install -q -U git+https://github.com/lvwerra/trl.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q transformers
!pip install -q tdqm torch
!pip install -q -U bitsandbytes
!pip install -q -U evaluate
# !pip install -q evaluate rouge_score
!pip install python-dotenv
!pip install peft
!pip install cohere
!pip install -q trl

In [3]:
import random
import torch
from datasets import (load_dataset,
                      Dataset)
from tqdm import tqdm
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          TrainingArguments,
                          AutoModelForSeq2SeqLM,
                          DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer,
                          Trainer,
                          GenerationConfig,
                          pipeline)
import evaluate


from datasets import get_dataset_split_names
import json
import pandas as pd

In [10]:
from data_handler.data_handler import GPT2DatasetHandler

In [11]:
DATASET_NAME = "EdinburghNLP/xsum"
dh = GPT2DatasetHandler(DATASET_NAME, tokenizer=None, data_size=100, push_to_hub=True, save_locally=False, rlaif=True)
data = dh.process_data()
tokenizer = dh.tokenizer

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Filter:   0%|          | 0/18 [00:00<?, ? examples/s]

ERROR:data_handler.data_handler:Couldn't push the data to HF. Is HF_UN  set in .env?
Traceback (most recent call last):
  File "/content/maet_pln/src/data_handler/data_handler.py", line 178, in save_dataset
    d.push_to_hub(f"{os.environ['HF_UN']}/{self.ID}_{k}")
  File "/usr/lib/python3.10/os.py", line 680, in __getitem__
    raise KeyError(key) from None
KeyError: 'HF_UN'


Map:   0%|          | 0/72 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1204 > 1024). Running this sequence through the model will result in indexing errors


Filter:   0%|          | 0/72 [00:00<?, ? examples/s]

ERROR:data_handler.data_handler:Couldn't push the data to HF. Is HF_UN  set in .env?
Traceback (most recent call last):
  File "/content/maet_pln/src/data_handler/data_handler.py", line 178, in save_dataset
    d.push_to_hub(f"{os.environ['HF_UN']}/{self.ID}_{k}")
  File "/usr/lib/python3.10/os.py", line 680, in __getitem__
    raise KeyError(key) from None
KeyError: 'HF_UN'
ERROR:data_handler.data_handler:Couldn't push the data to HF. Is HF_UN  set in .env?
Traceback (most recent call last):
  File "/content/maet_pln/src/data_handler/data_handler.py", line 178, in save_dataset
    d.push_to_hub(f"{os.environ['HF_UN']}/{self.ID}_{k}")
  File "/usr/lib/python3.10/os.py", line 680, in __getitem__
    raise KeyError(key) from None
KeyError: 'HF_UN'


In [12]:
sft_train_data, rlaif_data, val_data = data
print([len(dataset) for dataset in data])

[17, 62, 9]


In [7]:
### SFT
from sft import SFT
from model_builder import GPT2ModelBuilder
import torch
torch.device("cuda")



device(type='cuda')

In [8]:
# SET UP BASE MODEL
save_dir = "sft-gpt2-xsum-0504"
base_model_id = "openai-community/gpt2-medium"
model_builder = GPT2ModelBuilder(model_id=base_model_id, tokenizer=tokenizer)
base_model = model_builder.base_model
stf_trainer = SFT(save_dir=save_dir,
              tokenizer=tokenizer,
              base_model=model_builder.base_model,
              train_dataset=sft_train_data)

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
torch.cuda.empty_cache()
stf_trainer.train_model()
stf_trainer.push_model_to_hub()
torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1834: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case jth500/sft-gpt2-xsum-0504).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
7,84.697000
14,56.404400
21,9.912400
28,4.185400


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

# RLAIF

In [17]:
from rlaif import RLAIF

torch.cuda.empty_cache()
sft_model = "jth500/" + save_dir
rlaif_model = "jth500/ + save_dir" + "rl"
rlaif_trainer = RLAIF(sft_model, tokenizer, rlaif_model, rlaif_data)

In [18]:
bm = rlaif_trainer.load_base_model()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1059: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [19]:
import warnings

# Filter warnings by message content
warnings.filterwarnings("ignore", message="A <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'> model is loaded from")
warnings.filterwarnings("ignore", message="The attention mask and the pad token id were not set.")
warnings.filterwarnings("ignore", message="Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.")
warnings.filterwarnings("ignore", message="A decoder-only architecture is being used, but right-padding was detected!")
warnings.filterwarnings("ignore", message="Your text contains a trailing whitespace, which has been trimmed to ensure high quality generations.")

In [ ]:
out = rlaif_trainer.train_model()

In [22]:
ref_ppo_delta, returns_mean, kl, loss = out

In [24]:
rlaif_trainer.base_model.push_to_hub('jth500/rlaif_gpt_0504')

adapter_model.safetensors:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jth500/rlaif_gpt_0504/commit/668c6ca497a07178059aca1091667e537ce216fb', commit_message='Upload model', commit_description='', oid='668c6ca497a07178059aca1091667e537ce216fb', pr_url=None, pr_revision=None, pr_num=None)

# Inference

In [15]:
import os
os.environ['COHERE_PROD_API_KEY'] = userdata.get('CO_API_KEY')


from  model_builder import GPT2ModelBuilder
from transformers import pipeline

In [16]:

rlaif_model_id = "jth500/rlaif_gpt_0504"
summarizer = pipeline('text-generation', model=rlaif_model_id, tokenizer=tokenizer)

In [18]:
TEMPLATE = (
    """{input} TL;DR:"""
)
prompts = [TEMPLATE.format(input=i) for i in val_data['input']]

In [19]:
responses = []
summaries = []
indices = []

for i, p in enumerate(tqdm(prompts)):
  try:
    r = summarizer(p, max_length=10000, truncation=True)
    responses.append(r[0]['generated_text'])
    summaries.append(val_data[i]['output'])
    indices.append(i)
  except:
    pass

 44%|████▍     | 4/9 [01:10<01:17, 15.47s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (1024). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.
100%|██████████| 9/9 [02:46<00:00, 18.47s/it]


In [21]:
df = pd.DataFrame({'summary_pred': responses, 'summary_actual': summaries, 'index': indices,
                   'input': val_data[indices]['input']})
df.head()

,summary_pred,summary_actual,index,input
0,Pte Paul Wilkinson said he asked Pte Cheryl Ja...,A boyfriend of a young soldier who died at Dee...,0,Pte Paul Wilkinson said he asked Pte Cheryl Ja...
1,Big oil producing nations meet this weekend in...,(Close):Wall Street markets fell on Friday as ...,1,Big oil producing nations meet this weekend in...
2,"Reus, 27, missed the 2014 World Cup and has ha...",Borussia Dortmund's Marco Reus has been omitte...,2,"Reus, 27, missed the 2014 World Cup and has ha..."
3,"Jardim, in charge since 2014, described the la...",Monaco boss Leonardo Jardim has been rewarded ...,3,"Jardim, in charge since 2014, described the la..."
4,On-loan striker Holman opened his account on h...,Dan Holman's brace helped Cheltenham to an eas...,5,On-loan striker Holman opened his account on h...
